In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from tqdm import tqdm
tqdm.pandas()
from dateutil.relativedelta import relativedelta
pd.set_option("display.max_columns", None)

# Date format = "%Y-%m-%d" @ datetime type

## BALANCE CALCULATIONS

### 1 READING DATA AND CLARIFYING DATA TYPES 1

In [2]:
mainDirect0 = pd.read_csv("Main_Director v020924.csv")    # Manager level data from 2006 to 2024

C:\Users\SHIVAM\AppData\Local\Temp\ipykernel_18048\952982046.py:1: DtypeWarning: Columns (28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  mainDirect0 = pd.read_csv("Main_Director v020924.csv")    # Manager level data from 2006 to 2024


In [3]:
mainDirect = mainDirect0.loc[ pd.isnull(mainDirect0["Cessation Date"]) == True].copy().reset_index(drop = True)
mainDirect

,Unnamed: 0,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till
0,0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,14-MAY-1960,M,INDIA,24-JUL-2002,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaN,NaN,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,15-DEC-1939,M,INDIA,26-DEC-2001,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,NaN,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3MINDIA,3M INDIA LTD.,2006-03-31,P00033331,MR.,BALASUBRAMANIAM,SOMASKANDAN,IYER,13-OCT-1949,M,INDIA,26-DEC-2001,NONE,GRADUATE: DIPLOMA-MANAGERIAL ACCOUNTING,"GRADUATE: FELLOW-MEMBER,INSTITUTE OF COMPANY S...","POST GRADUATE: M.COM.,UNIVERSITY OF MUMBAI,MUMBAI","POST GRADUATE: M.L.,UNIVERSITY OF MUMBAI,MUMBAI","POST GRADUATE: MEMBER,ALL INDIA MANAGEMENT ASS...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MR. B.S. IYER, 62, IS A POST GRADUATE IN COMME...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3PLAND,3P LAND HOLDINGS LTD.,2006-03-31,P00022755,LATE MR.,SHAHRUKH,MINOCHER,CONTRACTOR,19-MAR-1931,M,NaN,31-DEC-1990,NaN,"GRADUATE: GRADUATE,A.T.A.,INDIA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MR.CONTRACTOR HAS BEEN ASSOCIATED WITH THE COM...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,3PLAND,3P LAND HOLDINGS LTD.,2006-03-31,P00012704,MR.,ATUL,SHANTIKUMAR,DAYAL,21-SEP-1948,M,INDIA,10-DEC-1990,NONE,GRADUATE: B.SC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MR.A. S. DAYAL, A LEGAL LUMINARY IN MUMBAI HAS...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228550,446419,ZYDUSWELL,ZYDUS WELLNESS LTD.,2024-03-31,P00043398,MR.,KULIN,SANJAY,LAL,13-AUG-1985,M,INDIA,18-NOV-2016,NONE,"GRADUATE: B.SC.-ELECTRICAL ENGINEERING,STANFOR...","POST GRADUATE: M.B.A.,HARVARD BUSINESS SCHOOL,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"CONSUMER DISCRETIONARY,CORPORATE SOCIAL RESPON...",NaN,NaN,"MR. KULIN LALBHAI, AGED 31 YEARS HOLDS AN MBA ...",NO,NON-EXECUTIVE DIRECTOR,YES,"EXECUTIVE DIRECTOR, ARVIND LTD.",ARVIND FASHIONS LTD.,ARVIND LTD.,ARVIND SMARTSPACES LTD.,ZYDUS WELLNESS LTD.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17-NOV-2026
228551,446420,ZYDUSWELL,ZYDUS WELLNESS LTD.,2024-03-31,P00027471,MR.,VISHNU,RAJU SRI,NANDYALA,20-DEC-1973,M,INDIA,11-MAR-2019,NONE,"GRADUATE: B.E.-CHEMICAL,OSMANIA UNIVERSITY,TEL...",POST GRADUATE: EXECUTIVE M.B.A.-OWNER,"PRESIDENT MANAGEMENT

In [4]:
# Mixed Data Type problematic columns

mixedDataColNum = [28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45] 
mixedDataCol = mainDirect.columns[mixedDataColNum]
mixedDataCol

Index(['Promoter Director (Yes/No)', 'Position on Board',
       'Independent (Yes/No)', 'Occupation', 'Other Directorship 1',
       'Other Directorship 2', 'Other Directorship 3', 'Other Directorship 4',
       'Other Directorship 5', 'Other Directorship 6', 'Other Directorship 7',
       'Other Directorship 8', 'Other Directorship 9', 'Other Directorship 10',
       'Other Directorship 11', 'Other Directorship 12',
       'Other Directorship 13', 'Other Directorship 14'],
      dtype='object')

In [5]:
# Mentioning key data types

mainDirect[mixedDataCol] = mainDirect[mixedDataCol].astype(str)

mainDirect["Cessation Reason"] = mainDirect["Cessation Reason"].astype(str)

mainDirect["AsOnDate"] = pd.to_datetime( mainDirect["AsOnDate"], format = "%Y-%m-%d" )
mainDirect["Date of Birth"] = pd.to_datetime( mainDirect["Date of Birth"], format = "%d-%b-%Y" )
mainDirect["Appointment Date"] = pd.to_datetime( mainDirect["Appointment Date"], format = "%d-%b-%Y" )
mainDirect["Cessation Date"] = pd.to_datetime( mainDirect["Cessation Date"], format = "%d-%b-%Y", errors = "coerce" )
mainDirect["Tenure Valid till"] = pd.to_datetime( mainDirect["Tenure Valid till"], format = "%d-%b-%Y", errors = "coerce" )


In [6]:
mainDirect = mainDirect.drop( labels = ['Unnamed: 0'], axis = 1 )
mainDirect

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaT,nan,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT
2,3MINDIA,3M INDIA LTD.,2006-03-31,P00033331,MR.,BALASUBRAMANIAM,SOMASKANDAN,IYER,1949-10-13,M,INDIA,2001-12-26,NONE,GRADUATE: DIPLOMA-MANAGERIAL ACCOUNTING,"GRADUATE: FELLOW-MEMBER,INSTITUTE OF COMPANY S...","POST GRADUATE: M.COM.,UNIVERSITY OF MUMBAI,MUMBAI","POST GRADUATE: M.L.,UNIVERSITY OF MUMBAI,MUMBAI","POST GRADUATE: MEMBER,ALL INDIA MANAGEMENT ASS...",NaN,NaN,NaN,NaN,NaN,NaN,NaT,nan,"MR. B.S. IYER, 62, IS A POST GRADUATE IN COMME...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT
3,3PLAND,3P LAND HOLDINGS LTD.,2006-03-31,P00022755,LATE MR.,SHAHRUKH,MINOCHER,CONTRACTOR,1931-03-19,M,NaN,1990-12-31,NaN,"GRADUATE: GRADUATE,A.T.A.,INDIA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,nan,MR.CONTRACTOR HAS BEEN ASSOCIATED WITH THE COM...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT
4,3PLAND,3P LAND HOLDINGS LTD.,2006-03-31,P00012704,MR.,ATUL,SHANTIKUMAR,DAYAL,1948-09-21,M,INDIA,1990-12-10,NONE,GRADUATE: B.SC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,nan,"MR.A. S. DAYAL, A LEGAL LUMINARY IN MUMBAI HAS...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228550,ZYDUSWELL,ZYDUS WELLNESS LTD.,2024-03-31,P00043398,MR.,KULIN,SANJAY,LAL,1985-08-13,M,INDIA,2016-11-18,NONE,"GRADUATE: B.SC.-ELECTRICAL ENGINEERING,STANFOR...","POST GRADUATE: M.B.A.,HARVARD BUSINESS SCHOOL,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"CONSUMER DISCRETIONARY,CORPORATE SOCIAL RESPON...",NaT,nan,"MR. KULIN LALBHAI, AGED 31 YEARS HOLDS AN MBA ...",NO,NON-EXECUTIVE DIRECTOR,YES,"EXECUTIVE DIRECTOR, ARVIND LTD.",ARVIND FASHIONS LTD.,ARVIND LTD.,ARVIND SMARTSPACES LTD.,ZYDUS WELLNESS LTD.,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,2026-11-17
228551,ZYDUSWELL,ZYDUS WELLNESS LTD.,2024-03-31,P00027471,MR.,VISHNU,RAJU SRI,NANDYALA,1973-12-20,M,INDIA,2019-03-11,NONE,"GRADUATE: B.E.-CHEMICAL,OSMANIA UNIVERSITY,TEL...",POST GRADUATE: EXECUTIVE M.B.A.-OWNER,"PRESIDENT MANAGEMENT PROGRAM,HARVARD BUSINESS ...",NaN,NaN,NaN,NaN,NaN,NaN

### 1 INDEP POPULATING 1

In [7]:
# Slicing the dataframe for eefficiency

balDirect = mainDirect[["Symbol", "AsOnDate", "Person Code", "Appointment Date", "Cessation Date", "Independent (Yes/No)"]]
balDirect = balDirect.sort_values(by = ["Symbol", "Person Code", "AsOnDate"]).set_index(["Symbol", "Person Code", "AsOnDate"], drop = False)

In [8]:
# IsIndep populating based on pre existing column as a dummy

balDirect.insert(column = "Indep", loc = 3, value = "U")
balDirect.loc[(balDirect["Independent (Yes/No)"] == "YES"), "Indep"] = "Y"
balDirect.loc[(balDirect["Independent (Yes/No)"] == "NO"), "Indep"] = "N"
balDirect = balDirect.drop("Independent (Yes/No)", axis = 1)

In [9]:
balDirect

Symbol   AsOnDate Person Code Indep  \
Symbol    Person Code AsOnDate                                             
20MICRONS P00004449   2024-03-31  20MICRONS 2024-03-31   P00004449     Y   
          P00027633   2009-03-31  20MICRONS 2009-03-31   P00027633     U   
                      2010-03-31  20MICRONS 2010-03-31   P00027633     U   
                      2011-03-31  20MICRONS 2011-03-31   P00027633     U   
                      2012-03-31  20MICRONS 2012-03-31   P00027633     U   
...                                     ...        ...         ...   ...   
ZYLOG     P00075400   2018-03-31      ZYLOG 2018-03-31   P00075400     Y   
          P00080494   2017-03-31      ZYLOG 2017-03-31   P00080494     Y   
                      2018-03-31      ZYLOG 2018-03-31   P00080494     Y   
          P00081419   2017-03-31      ZYLOG 2017-03-31   P00081419     Y   
                      2018-03-31      ZYLOG 2018-03-31   P00081419     Y   

                                 Appointment Date Cessation Date  
Symbol    Person Code AsOnDate                                    
20MICRONS P00004449   2024-03-31       2023-05-16            NaT  
          P00027633   2009-03-31       2007-10-13            NaT  
                      2010-03-31       2007-10-13            NaT  
                      2011-03-31       2007-10-13            NaT  
                      2012-03-31       2007-10-13            NaT  
...                                           ...            ...  
ZYLOG     P00075400   2018-03-31       2015-11-25            NaT  
          P00080494   2017-03-31       2016-11-23            NaT  
                      2018-03-31       2016-11-23            NaT  
          P00081419   2017-03-31       2017-02-10            NaT  
                      2018-03-31       2017-02-10            NaT  

[228555 rows x 6 columns]

### 1 PREVIOUS LAST SERVED 2

In [10]:
# Which date does this person appear last in that company

def PrevLastServed(frame):
    frameOut = pd.DataFrame( columns = ["AsOnDate", "PrevLastServed"] )
    frameOut["AsOnDate"] = frame["AsOnDate"]
    frameOut["PrevLastServed"] = frame["AsOnDate"].shift( periods = 1, axis = 0 )
    return frameOut

In [11]:
prevLastServed = balDirect.reset_index(drop = True).groupby(
    ["Symbol", "Person Code"] ).progress_apply( PrevLastServed )

100%|███████████████████████████████████████████████████████████████████████████| 43722/43722 [04:44<00:00, 153.58it/s]


In [12]:
prevLastServed.reset_index( drop = False ).drop(["level_2"], axis =1 )

,Symbol,Person Code,AsOnDate,PrevLastServed
0,20MICRONS,P00004449,2024-03-31,NaT
1,20MICRONS,P00027633,2009-03-31,NaT
2,20MICRONS,P00027633,2010-03-31,2009-03-31
3,20MICRONS,P00027633,2011-03-31,2010-03-31
4,20MICRONS,P00027633,2012-03-31,2011-03-31
...,...,...,...,...
228550,ZYLOG,P00075400,2018-03-31,2017-03-31
228551,ZYLOG,P00080494,2017-03-31,NaT
228552,ZYLOG,P00080494,2018-03-31,2017-03-31
228553,ZYLOG,P00081419,2017-03-31,NaT


In [13]:
balDirect2 = pd.merge( balDirect.reset_index(drop = True), prevLastServed.reset_index( drop = False ).drop(["level_2"], axis =1 ), on = ["Symbol", "Person Code", "AsOnDate"])

In [14]:
balDirect2

,Symbol,AsOnDate,Person Code,Indep,Appointment Date,Cessation Date,PrevLastServed
0,20MICRONS,2024-03-31,P00004449,Y,2023-05-16,NaT,NaT
1,20MICRONS,2009-03-31,P00027633,U,2007-10-13,NaT,NaT
2,20MICRONS,2010-03-31,P00027633,U,2007-10-13,NaT,2009-03-31
3,20MICRONS,2011-03-31,P00027633,U,2007-10-13,NaT,2010-03-31
4,20MICRONS,2012-03-31,P00027633,U,2007-10-13,NaT,2011-03-31
...,...,...,...,...,...,...,...
228550,ZYLOG,2018-03-31,P00075400,Y,2015-11-25,NaT,2017-03-31
228551,ZYLOG,2017-03-31,P00080494,Y,2016-11-23,NaT,NaT
228552,ZYLOG,2018-03-31,P00080494,Y,2016-11-23,NaT,2017-03-31
228553,ZYLOG,2017-03-31,P00081419,Y,2017-02-10,NaT,NaT


### 2 CLOSING BALANCE INITIALIZATION AND CUMULATIVE SUM USAGE 3

In [15]:
# Instantiating and assigning Closing Balance variables
# We use closing balance here, as these variables are the ones that capture data from the current row. 
# Opening bal variables bring data from previous rows.

balDirect2["CloBalUnc"] = 0
balDirect2["CloBalIndep"] = 0
balDirect2["CloBalNonIndep"] = 0
balDirect2

,Symbol,AsOnDate,Person Code,Indep,Appointment Date,Cessation Date,PrevLastServed,CloBalUnc,CloBalIndep,CloBalNonIndep
0,20MICRONS,2024-03-31,P00004449,Y,2023-05-16,NaT,NaT,0,0,0
1,20MICRONS,2009-03-31,P00027633,U,2007-10-13,NaT,NaT,0,0,0
2,20MICRONS,2010-03-31,P00027633,U,2007-10-13,NaT,2009-03-31,0,0,0
3,20MICRONS,2011-03-31,P00027633,U,2007-10-13,NaT,2010-03-31,0,0,0
4,20MICRONS,2012-03-31,P00027633,U,2007-10-13,NaT,2011-03-31,0,0,0
...,...,...,...,...,...,...,...,...,...,...
228550,ZYLOG,2018-03-31,P00075400,Y,2015-11-25,NaT,2017-03-31,0,0,0
228551,ZYLOG,2017-03-31,P00080494,Y,2016-11-23,NaT,NaT,0,0,0
228552,ZYLOG,2018-03-31,P00080494,Y,2016-11-23,NaT,2017-03-31,0,0,0
228553,ZYLOG,2017-03-31,P00081419,Y,2017-02-10,NaT,NaT,0,0,0


In [16]:
# cumulative sum to capture cumulative years of Independent status within a company for a person
# initiate cumsum values with 1 wrt their correspondant independent status, these will cumulative summed over later

balDirect2["cumsum_U"] = 0
balDirect2["cumsum_Y"] = 0
balDirect2["cumsum_N"] = 0

balDirect2.loc[ balDirect2["Indep"] == "U","cumsum_U"] = 1
balDirect2.loc[ balDirect2["Indep"] == "Y","cumsum_Y"] = 1
balDirect2.loc[ balDirect2["Indep"] == "N","cumsum_N"] = 1

In [17]:
# Assigns Closing Balance Unclear with number of days of term before any record. Since there are no records,
# these days are under unclear independent status

# Here, this variable is not showcasing the status as of closing current year, but is showing opening current year.
# The current years" term days will be added later and opening balance will be inferred afterwards.

# cumulative sum function works on cumsum columns

def CloBalUncInitial(frame):
    frame["CloBalUnc"] += (frame.loc[:, "AsOnDate"].min() - frame["Appointment Date"]).dt.days

    frame["cumsum_U"] = np.cumsum(frame["cumsum_U"])
    frame["cumsum_Y"] = np.cumsum(frame["cumsum_Y"])
    frame["cumsum_N"] = np.cumsum(frame["cumsum_N"])
    return (frame)

In [18]:
balDirect3 = balDirect2.reset_index(drop = True).groupby(
    ["Symbol", "Person Code"] ).progress_apply( CloBalUncInitial )

100%|████████████████████████████████████████████████████████████████████████████| 43722/43722 [08:53<00:00, 82.02it/s]


### 3 CLOSING BALANCE FINALIZATION 3

In [19]:
# Adds 365 days based on current rows cumulative terms as dictated by cumsum columns
# cumsum carries forward the terms throughout the symbolxPCode combination

  
balDirect3.loc[ (balDirect3["cumsum_U"] > 0 ) , "CloBalUnc" ] += (balDirect3["cumsum_U"]) * 365   
balDirect3.loc[ (balDirect3["cumsum_Y"] > 0 ) , "CloBalIndep" ] += (balDirect3["cumsum_Y"]) * 365  
balDirect3.loc[ (balDirect3["cumsum_N"] > 0 ) , "CloBalNonIndep" ] += (balDirect3["cumsum_N"]) * 365
        
balDirect3["CloBalTotal"] = 0
balDirect3["CloBalTotal"] = balDirect3["CloBalUnc"] + balDirect3["CloBalIndep"] + balDirect3["CloBalNonIndep"]

In [20]:
balDirect3

Symbol   AsOnDate Person Code Indep  \
Symbol    Person Code                                                  
20MICRONS P00004449   0       20MICRONS 2024-03-31   P00004449     Y   
          P00027633   1       20MICRONS 2009-03-31   P00027633     U   
                      2       20MICRONS 2010-03-31   P00027633     U   
                      3       20MICRONS 2011-03-31   P00027633     U   
                      4       20MICRONS 2012-03-31   P00027633     U   
...                                 ...        ...         ...   ...   
ZYLOG     P00075400   228550      ZYLOG 2018-03-31   P00075400     Y   
          P00080494   228551      ZYLOG 2017-03-31   P00080494     Y   
                      228552      ZYLOG 2018-03-31   P00080494     Y   
          P00081419   228553      ZYLOG 2017-03-31   P00081419     Y   
                      228554      ZYLOG 2018-03-31   P00081419     Y   

                             Appointment Date Cessation Date PrevLastServed  \
Symbol    Person Code                                                         
20MICRONS P00004449   0            2023-05-16            NaT            NaT   
          P00027633   1            2007-10-13            NaT            NaT   
                      2            2007-10-13            NaT     2009-03-31   
                      3            2007-10-13            NaT     2010-03-31   
                      4            2007-10-13            NaT     2011-03-31   
...                                       ...            ...            ...   
ZYLOG     P00075400   228550       2015-11-25            NaT     2017-03-31   
          P00080494   228551       2016-11-23            NaT            NaT   
                      228552       2016-11-23            NaT     2017-03-31   
          P00081419   228553       2017-02-10            NaT            NaT   
                      228554       2017-02-10            NaT     2017-03-31   

                              CloBalUnc  CloBalIndep  CloBalNonIndep  \
Symbol    Person Code                                                  
20MICRONS P00004449   0           320.0          365               0   
          P00027633   1           900.0            0               0   
                      2          1265.0            0               0   
                      3          1630.0            0               0   
                      4          1995.0            0               0   
...                                 ...          ...             ...   
ZYLOG     P00075400   228550      127.0         1095               0   
          P00080494   228551      128.0          365               0   
                      228552      128.0          730               0   
          P00081419   228553       49.0          365               0   
                      228554       49.0          730               0   

                              cumsum_U  cumsum_Y  cumsum_N  CloBalTotal  
Symbol    Person Code                                                    
20MICRONS P00004449   0              0         1         0        685.0  
          P00027633   1              1         0         0        900.0  
                      2              2         0         0       1265.0  
                      3              3         0         0       1630.0  
                      4              4         0         0       1995.0  
...                                ...       ...       ...          ...  
ZYLOG     P00075400   228550         0         3         0       1222.0  
          P00080494   228551         0         1         0        493.0  
                      228552         0         2         0        858.0  
          P00081419   228553         0         1         0        414.0  
                      228554         0         2         0        779.0  

[228555 rows x 14 columns]

### 3 OPENING BALANCE AND YOE 5

In [21]:
# Opening Blance Claculations

balDirect3["OpBalUnc"] = 0
balDirect3["OpBalIndep"] = 0
balDirect3["OpBalNonIndep"] = 0
balDirect3["OpBalTotal"] = 0

balDirect3["OpBalUnc"] = balDirect3["CloBalUnc"]
balDirect3["OpBalIndep"] = balDirect3["CloBalIndep"]
balDirect3["OpBalNonIndep"] = balDirect3["CloBalNonIndep"]

balDirect3.loc[ balDirect3["Indep"] == "U","OpBalUnc"] -= (365)
balDirect3.loc[ balDirect3["Indep"] == "Y","OpBalIndep"] -= (365)
balDirect3.loc[ balDirect3["Indep"] == "N","OpBalNonIndep"] -= (365)
balDirect3["OpBalTotal"] = 0
balDirect3["OpBalTotal"] = balDirect3["OpBalUnc"] + balDirect3["OpBalIndep"] + balDirect3["OpBalNonIndep"]

In [22]:
# Rearranging the Datafile

balDirect3cols = balDirect3.columns
balDirect3cols_rearranged = ['Symbol',  'Person Code', 'AsOnDate', 'PrevLastServed', 'Indep', 'Appointment Date', 'Cessation Date',  
                             'cumsum_U', 'cumsum_Y', 'cumsum_N',
                             'OpBalUnc', 'OpBalIndep', 'OpBalNonIndep', 'OpBalTotal',
                             'CloBalUnc', 'CloBalIndep', 'CloBalNonIndep','CloBalTotal',
                            ]
balDirect4 = balDirect3[balDirect3cols_rearranged].reset_index(drop = True)
balDirect4

,Symbol,Person Code,AsOnDate,PrevLastServed,Indep,Appointment Date,Cessation Date,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal
0,20MICRONS,P00004449,2024-03-31,NaT,Y,2023-05-16,NaT,0,1,0,320.0,0,0,320.0,320.0,365,0,685.0
1,20MICRONS,P00027633,2009-03-31,NaT,U,2007-10-13,NaT,1,0,0,535.0,0,0,535.0,900.0,0,0,900.0
2,20MICRONS,P00027633,2010-03-31,2009-03-31,U,2007-10-13,NaT,2,0,0,900.0,0,0,900.0,1265.0,0,0,1265.0
3,20MICRONS,P00027633,2011-03-31,2010-03-31,U,2007-10-13,NaT,3,0,0,1265.0,0,0,1265.0,1630.0,0,0,1630.0
4,20MICRONS,P00027633,2012-03-31,2011-03-31,U,2007-10-13,NaT,4,0,0,1630.0,0,0,1630.0,1995.0,0,0,1995.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228550,ZYLOG,P00075400,2018-03-31,2017-03-31,Y,2015-11-25,NaT,0,3,0,127.0,730,0,857.0,127.0,1095,0,1222.0
228551,ZYLOG,P00080494,2017-03-31,NaT,Y,2016-11-23,NaT,0,1,0,128.0,0,0,128.0,128.0,365,0,493.0
228552,ZYLOG,P00080494,2018-03-31,2017-03-31,Y,2016-11-23,NaT,0,2,0,128.0,365,0,493.0,128.0,730,0,858.0
228553,ZYLOG,P00081419,2017-03-31,NaT,Y,2017-02-10,NaT,0,1,0,49.0,0,0,49.0,49.0,365,0,414.0


In [23]:
# YOE
balDirect4.insert(len(balDirect4.columns), column = 'YOE', value = 0)
balDirect4['YOE'] = (balDirect4['AsOnDate'] - balDirect4['Appointment Date']).dt.days

In [24]:
# YOE MAX
def YOE(frame):
    return frame['YOE'].max()

In [25]:
YOEmax = balDirect4.groupby(['Person Code', 'AsOnDate']).progress_apply(YOE)

100%|████████████████████████████████████████████████████████████████████████| 181097/181097 [00:20<00:00, 9013.60it/s]


In [26]:
YOEmax = YOEmax.reset_index()
YOEmax

,Person Code,AsOnDate,0
0,P00000005,2006-03-31,8612.0
1,P00000005,2007-03-31,8977.0
2,P00000005,2008-03-31,9343.0
3,P00000005,2009-03-31,9708.0
4,P00000005,2010-03-31,10073.0
...,...,...,...
181092,P00118177,2023-03-31,198.0
181093,P00118177,2024-03-31,564.0
181094,P00118252,2024-03-31,4.0
181095,P00118305,2024-03-31,246.0


In [27]:
balDirect5 = balDirect4.merge(YOEmax, on = ['Person Code', 'AsOnDate'])
balDirect5 = balDirect5.rename(columns = {0 : "YOEmax"})
balDirect5.sort_values(by = ["Person Code", "Symbol", "AsOnDate"])

,Symbol,Person Code,AsOnDate,PrevLastServed,Indep,Appointment Date,Cessation Date,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax
8727,AEGISCHEM,P00000005,2006-03-31,NaT,U,1982-09-01,NaT,1,0,0,8612.0,0,0,8612.0,8977.0,0,0,8977.0,8612.0,8612.0
8728,AEGISCHEM,P00000005,2007-03-31,2006-03-31,U,1982-09-01,NaT,2,0,0,8977.0,0,0,8977.0,9342.0,0,0,9342.0,8977.0,8977.0
8729,AEGISCHEM,P00000005,2008-03-31,2007-03-31,U,1982-09-01,NaT,3,0,0,9342.0,0,0,9342.0,9707.0,0,0,9707.0,9343.0,9343.0
8730,AEGISCHEM,P00000005,2009-03-31,2008-03-31,U,1982-09-01,NaT,4,0,0,9707.0,0,0,9707.0,10072.0,0,0,10072.0,9708.0,9708.0
8731,AEGISCHEM,P00000005,2010-03-31,2009-03-31,U,1982-09-01,NaT,5,0,0,10072.0,0,0,10072.0,10437.0,0,0,10437.0,10073.0,10073.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105005,HIGHWAYS,P00118177,2023-03-31,NaT,U,2022-09-14,NaT,1,0,0,198.0,0,0,198.0,563.0,0,0,563.0,198.0,198.0
105006,HIGHWAYS,P00118177,2024-03-31,2023-03-31,U,2022-09-14,NaT,2,0,0,563.0,0,0,563.0,928.0,0,0,928.0,564.0,564.0
148607,MAXESTATES,P00118252,2024-03-31,NaT,U,2024-03-27,NaT,1,0,0,4.0,0,0,4.0,369.0,0,0,369.0,4.0,4.0
78542,EPACK,P00118305,2024-03-31,NaT,U,2023-07-29,NaT,1,0,0,246.0,0,0,246.0,611.0,0,0,611.0,246.0,246.0


### 1 MERGE WITH MAIN 2

In [28]:
mainDirect2 = mainDirect.merge(balDirect5.reset_index(drop = True), on = ["Symbol", "Person Code", "AsOnDate", "Appointment Date", "Cessation Date"], how = "inner", validate = "1:1")

In [29]:
mainDirect2

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaT,nan,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,3289.0
2,3MINDIA,3M INDIA LTD.,2006-03-31,P00033331,MR.,BALASUBRAMANIAM,SOMASKANDAN,IYER,1949-10-13,M,INDIA,2001-12-26,NONE,GRADUATE: DIPLOMA-MANAGERIAL ACCOUNTING,"GRADUATE: FELLOW-MEMBER,INSTITUTE OF COMPANY S...","POST GRADUATE: M.COM.,UNIVERSITY OF MUMBAI,MUMBAI","POST GRADUATE: M.L.,UNIVERSITY OF MUMBAI,MUMBAI","POST GRADUATE: MEMBER,ALL INDIA MANAGEMENT ASS...",NaN,NaN,NaN,NaN,NaN,NaN,NaT,nan,"MR. B.S. IYER, 62, IS A POST GRADUATE IN COMME...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,1556.0
3,3PLAND,3P LAND HOLDINGS LTD.,2006-03-31,P00022755,LATE MR.,SHAHRUKH,MINOCHER,CONTRACTOR,1931-03-19,M,NaN,1990-12-31,NaN,"GRADUATE: GRADUATE,A.T.A.,INDIA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,nan,MR.CONTRACTOR HAS BEEN ASSOCIATED WITH THE COM...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,5569.0,0,0,5569.0,5934.0,0,0,5934.0,5569.0,5569.0
4,3PLAND,3P LAND HOLDINGS LTD.,2006-03-31,P00012704,MR.,ATUL,SHANTIKUMAR,DAYAL,1948-09-21,M,INDIA,1990-12-10,NONE,GRADUATE: B.SC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,nan,"MR.A. S. DAYAL, A LEGAL LUMINARY IN MUMBAI HAS...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,5590.0,0,0,5590.0,5955.0,0,0,5955.0,5590.0,5590.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228550,ZYDUSWELL,ZYDUS WELLNESS LTD.,2024-03-31,P00043398,MR.,KULIN,SANJAY,LAL,1985-08-13,M,INDIA,2016-11-18,NONE,"GRADUATE: B.SC.-ELECTRICAL ENGINEERING,STANFOR...","POST GRADUATE: M.B.A.,HARVARD BUSINESS SCHOOL,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"CONSUMER DISCRETIONARY,CORPORATE SOCIAL RESPON...",NaT,nan,"MR. KULIN L

## COMP COUNT CALCULATIONS

In [30]:
compCountColNum = [0,1,2,3,48,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]
compCountCol = mainDirect2.columns[compCountColNum]
compCountCol

Index(['Symbol', 'Company', 'AsOnDate', 'Person Code', 'Indep',
       'Other Directorship 1', 'Other Directorship 2', 'Other Directorship 3',
       'Other Directorship 4', 'Other Directorship 5', 'Other Directorship 6',
       'Other Directorship 7', 'Other Directorship 8', 'Other Directorship 9',
       'Other Directorship 10', 'Other Directorship 11',
       'Other Directorship 12', 'Other Directorship 13',
       'Other Directorship 14', 'Other Directorship 15'],
      dtype='object')

In [31]:
compDirect = mainDirect2[compCountCol]
compDirect

,Symbol,Company,AsOnDate,Person Code,Indep,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,U,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,U,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
2,3MINDIA,3M INDIA LTD.,2006-03-31,P00033331,U,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
3,3PLAND,3P LAND HOLDINGS LTD.,2006-03-31,P00022755,U,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
4,3PLAND,3P LAND HOLDINGS LTD.,2006-03-31,P00012704,U,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228550,ZYDUSWELL,ZYDUS WELLNESS LTD.,2024-03-31,P00043398,Y,ARVIND FASHIONS LTD.,ARVIND LTD.,ARVIND SMARTSPACES LTD.,ZYDUS WELLNESS LTD.,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
228551,ZYDUSWELL,ZYDUS WELLNESS LTD.,2024-03-31,P00027471,Y,AMARA RAJA ENERGY & MOBILITY LTD.,ZYDUS WELLNESS LTD.,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
228552,ZYDUSWELL,ZYDUS WELLNESS LTD.,2024-03-31,P00022618,N,ZYDUS LIFESCIENCES LTD.,ZYDUS WELLNESS LTD.,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
228553,ZYDUSWELL,ZYDUS WELLNESS LTD.,2024-03-31,P00013240,N,ZYDUS LIFESCIENCES LTD.,ZYDUS WELLNESS LTD.,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN


### 1 PAST DIRECTORSHIPS 1

In [32]:
# Per person, per AsOnDate, we get all the previous Directorships along with their Indep status

def PastDirect(frame):
    frameOut = pd.DataFrame(index = frame.index, columns = ['AsOnDate','Past Directorships']) # Output Frame
    
    for i in range(len(frame)):                                                               # iterating through the length of input frame
        Date = frame.iloc[i]['AsOnDate']                                                      # Date = AsOnDate for that iteration
        framePast = frame.loc[ frame['AsOnDate'] < Date , ['Symbol', 'Indep']]                # Creating a frame with date before current Date
        pastDirectors = list(zip(framePast['Symbol'], framePast['Indep']))                    # list of tuples of the company and indep status in the past_frame
        pastDirUnique = tuple(set(pastDirectors))                                             # converting the above to a set, to remove duplicates, and then a list for easier use

        frameOut.at[ frame.index[i], 'Past Directorships'] = pastDirUnique                    # putting the above data to the output frame
        frameOut.at[ frame.index[i], 'AsOnDate'] = Date

    return frameOut

In [33]:
pastDirect = compDirect.sort_values(by = ['Person Code', 'AsOnDate', 'Company']).groupby('Person Code').progress_apply(PastDirect)

100%|███████████████████████████████████████████████████████████████████████████| 31347/31347 [04:25<00:00, 118.19it/s]


In [34]:
pastDirect

AsOnDate Past Directorships
Person Code                                               
P00000005   116     2006-03-31 00:00:00                 ()
            6465    2007-03-31 00:00:00  ((AEGISCHEM, U),)
            14383   2008-03-31 00:00:00  ((AEGISCHEM, U),)
            23577   2009-03-31 00:00:00  ((AEGISCHEM, U),)
            33220   2010-03-31 00:00:00  ((AEGISCHEM, U),)
...                                 ...                ...
P00118177   200614  2023-03-31 00:00:00                 ()
            217134  2024-03-31 00:00:00   ((HIGHWAYS, U),)
P00118252   220704  2024-03-31 00:00:00                 ()
P00118305   215366  2024-03-31 00:00:00                 ()
P00118306   215362  2024-03-31 00:00:00                 ()

[228555 rows x 2 columns]

In [35]:
pastDirect2 = pastDirect.reset_index().drop( labels = ["level_1"], axis = 1 ).drop_duplicates().rename( {"Past Directorships":"AllPastDirect"}, axis = 1)

In [36]:
pastDirect2

,Person Code,AsOnDate,AllPastDirect
0,P00000005,2006-03-31 00:00:00,()
1,P00000005,2007-03-31 00:00:00,"((AEGISCHEM, U),)"
2,P00000005,2008-03-31 00:00:00,"((AEGISCHEM, U),)"
3,P00000005,2009-03-31 00:00:00,"((AEGISCHEM, U),)"
4,P00000005,2010-03-31 00:00:00,"((AEGISCHEM, U),)"
...,...,...,...
228550,P00118177,2023-03-31 00:00:00,()
228551,P00118177,2024-03-31 00:00:00,"((HIGHWAYS, U),)"
228552,P00118252,2024-03-31 00:00:00,()
228553,P00118305,2024-03-31 00:00:00,()


### 1 CURRENT DIRECTORSHIPS 1

In [37]:
otherDirectColNum = [31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]
otherDirectCol = mainDirect2.columns[otherDirectColNum]
otherDirectCol

Index(['Other Directorship 1', 'Other Directorship 2', 'Other Directorship 3',
       'Other Directorship 4', 'Other Directorship 5', 'Other Directorship 6',
       'Other Directorship 7', 'Other Directorship 8', 'Other Directorship 9',
       'Other Directorship 10', 'Other Directorship 11',
       'Other Directorship 12', 'Other Directorship 13',
       'Other Directorship 14', 'Other Directorship 15'],
      dtype='object')

In [38]:
# A = Current Directorships in other records
# B = Current Directorships in other directorships columns



# A if A in B
def CurrentDirectAinB(frame, cols): 
    # Ensure cols is a list of column names and that they exist in the DataFrame
    if not all(col in frame.columns for col in cols):
        raise ValueError("One or more columns in 'cols' are not in the DataFrame")

    # Create unique list of tuples from 'Company' and 'Indep' in the current group
    comp_stat = list(set(zip(frame.Company, frame.Indep)))
    
    # Initialize the new DataFrame
    
    frameOut = pd.DataFrame(index=frame.index, columns=['Current Direct'])
    
    # Populate 'Current Direct' column
    for i in range(len(frame)):
        # Get the value for each column - All other directorships from other directorships columns
        row_values = [str(frame[col].iloc[i]) for col in cols]
        # Check if any company from comp_stat is in row_values - If companies within the frame are listed in other directorships columns
        current_direct = tuple(
            item for item in comp_stat if item[0] in row_values
        )
        frameOut.loc[frame.index[i], 'Current Direct'] = current_direct
    
    return frameOut

In [39]:
currentDirectAinB = compDirect.groupby(['Person Code', 'AsOnDate']).progress_apply(CurrentDirectAinB, cols=otherDirectCol)

100%|█████████████████████████████████████████████████████████████████████████| 181097/181097 [19:19<00:00, 156.23it/s]


In [41]:
currentDirectAinB

Current Direct
Person Code AsOnDate                                                
P00000005   2006-03-31 116                                        ()
            2007-03-31 6465                                       ()
            2008-03-31 14383                                      ()
            2009-03-31 23577                                      ()
            2010-03-31 33220                                      ()
...                                                              ...
P00118177   2023-03-31 200614  ((HIGHWAYS INFRASTRUCTURE TRUST, U),)
            2024-03-31 217134  ((HIGHWAYS INFRASTRUCTURE TRUST, U),)
P00118252   2024-03-31 220704               ((MAX ESTATES LTD., U),)
P00118305   2024-03-31 215366             ((EPACK DURABLE LTD., U),)
P00118306   2024-03-31 215362             ((EPACK DURABLE LTD., U),)

[228555 rows x 1 columns]

In [42]:
noDataDates = ["2006-03-31", "2007-03-31", "2008-03-31", "2009-03-31", "2010-03-31", "2011-03-31", "2012-03-31"]
noDataDates = pd.to_datetime( noDataDates, format = "%Y-%m-%d")
                             
currentDirect2 = currentDirectAinB.reset_index( drop = False ).drop(["level_2"], axis = 1).drop_duplicates().reset_index( drop = True )
currentDirect3 = currentDirect2.loc[~currentDirect2.AsOnDate.isin(noDataDates)]
currentDirect3

,Person Code,AsOnDate,Current Direct
7,P00000005,2013-03-31,"((AEGIS LOGISTICS LTD., N),)"
8,P00000005,2014-03-31,"((AEGIS LOGISTICS LTD., N),)"
9,P00000005,2015-03-31,"((AEGIS LOGISTICS LTD., N),)"
10,P00000005,2016-03-31,"((AEGIS LOGISTICS LTD., N),)"
11,P00000005,2017-03-31,"((AEGIS LOGISTICS LTD., N),)"
...,...,...,...
181092,P00118177,2023-03-31,"((HIGHWAYS INFRASTRUCTURE TRUST, U),)"
181093,P00118177,2024-03-31,"((HIGHWAYS INFRASTRUCTURE TRUST, U),)"
181094,P00118252,2024-03-31,"((MAX ESTATES LTD., U),)"
181095,P00118305,2024-03-31,"((EPACK DURABLE LTD., U),)"


In [43]:
# A = Current Directorships in other records 

def CurrentDirectA(frame):
    cols=otherDirectCol

    # Create unique list of tuples from 'Company' and 'Indep' in the current group
    comp_stat = tuple(set(zip(frame.Company, frame.Indep)))

    return comp_stat

In [44]:
currentDirectA = compDirect.groupby(['Person Code', 'AsOnDate']).progress_apply(CurrentDirectA)

100%|████████████████████████████████████████████████████████████████████████| 181097/181097 [00:24<00:00, 7371.18it/s]


In [45]:
currentDirectA

Person Code  AsOnDate  
P00000005    2006-03-31             ((AEGIS LOGISTICS LTD., U),)
             2007-03-31             ((AEGIS LOGISTICS LTD., U),)
             2008-03-31             ((AEGIS LOGISTICS LTD., U),)
             2009-03-31             ((AEGIS LOGISTICS LTD., U),)
             2010-03-31             ((AEGIS LOGISTICS LTD., U),)
                                           ...                  
P00118177    2023-03-31    ((HIGHWAYS INFRASTRUCTURE TRUST, U),)
             2024-03-31    ((HIGHWAYS INFRASTRUCTURE TRUST, U),)
P00118252    2024-03-31                 ((MAX ESTATES LTD., U),)
P00118305    2024-03-31               ((EPACK DURABLE LTD., U),)
P00118306    2024-03-31               ((EPACK DURABLE LTD., U),)
Length: 181097, dtype: object

In [46]:
currentDirectA2 = currentDirectA.to_frame().reset_index( drop = False ).drop_duplicates().rename({0:"CurrDirectA"}, axis = 1).reset_index( drop = True )
currentDirectA2

,Person Code,AsOnDate,CurrDirectA
0,P00000005,2006-03-31,"((AEGIS LOGISTICS LTD., U),)"
1,P00000005,2007-03-31,"((AEGIS LOGISTICS LTD., U),)"
2,P00000005,2008-03-31,"((AEGIS LOGISTICS LTD., U),)"
3,P00000005,2009-03-31,"((AEGIS LOGISTICS LTD., U),)"
4,P00000005,2010-03-31,"((AEGIS LOGISTICS LTD., U),)"
...,...,...,...
181092,P00118177,2023-03-31,"((HIGHWAYS INFRASTRUCTURE TRUST, U),)"
181093,P00118177,2024-03-31,"((HIGHWAYS INFRASTRUCTURE TRUST, U),)"
181094,P00118252,2024-03-31,"((MAX ESTATES LTD., U),)"
181095,P00118305,2024-03-31,"((EPACK DURABLE LTD., U),)"


In [47]:
# comparison shows that there is a mismatch, which means that other directorships and current directorships may not be same, Ill check it outside in excel
# currentDirect3.to_csv("Check1.csv")
# currentDirectCOMPARE3.to_csv("Check2.csv")

In [48]:
# A + B = Current Directorships in other directorships columns + Current Directorships in other directorships columns

def CurrentDirectAB(frame):
    cols=otherDirectCol

    # Create unique list of tuples from 'Company' and 'Indep' in the current group
    comp_stat = list(set(zip(frame.Company, frame.Indep)))

    for i in range(len(frame)):
        # Create a list of companies in other direct columns
        other_direct = set(str(frame[col].iloc[i]) for col in cols)
        other_direct2 = tuple(x for x in other_direct if x==x)
    
    other_direct3 = list(set(zip([x for x in other_direct2], "U")))
    
    current_direct = tuple(comp_stat + other_direct3)

    current_direct2 = tuple(x for x in current_direct if x[0] != "nan" )
    
    return current_direct2

In [49]:
currentDirectAB = compDirect.groupby(['Person Code', 'AsOnDate']).progress_apply(CurrentDirectAB)

100%|████████████████████████████████████████████████████████████████████████| 181097/181097 [02:32<00:00, 1188.45it/s]


In [50]:
currentDirectAB

Person Code  AsOnDate  
P00000005    2006-03-31                      ((AEGIS LOGISTICS LTD., U),)
             2007-03-31                      ((AEGIS LOGISTICS LTD., U),)
             2008-03-31                      ((AEGIS LOGISTICS LTD., U),)
             2009-03-31                      ((AEGIS LOGISTICS LTD., U),)
             2010-03-31                      ((AEGIS LOGISTICS LTD., U),)
                                                ...                      
P00118177    2023-03-31             ((HIGHWAYS INFRASTRUCTURE TRUST, U),)
             2024-03-31             ((HIGHWAYS INFRASTRUCTURE TRUST, U),)
P00118252    2024-03-31    ((MAX ESTATES LTD., U), (MAX ESTATES LTD., U))
P00118305    2024-03-31                        ((EPACK DURABLE LTD., U),)
P00118306    2024-03-31                        ((EPACK DURABLE LTD., U),)
Length: 181097, dtype: object

In [51]:
# currentDirectFINAL.to_csv("Check3.csv")

In [52]:
currentDirectAB2 = currentDirectAB.to_frame().reset_index( drop = False ).drop_duplicates().rename({0:"CurrDirectAB"}, axis = 1).reset_index( drop = True )

In [53]:
currentDirectAB2

,Person Code,AsOnDate,CurrDirectAB
0,P00000005,2006-03-31,"((AEGIS LOGISTICS LTD., U),)"
1,P00000005,2007-03-31,"((AEGIS LOGISTICS LTD., U),)"
2,P00000005,2008-03-31,"((AEGIS LOGISTICS LTD., U),)"
3,P00000005,2009-03-31,"((AEGIS LOGISTICS LTD., U),)"
4,P00000005,2010-03-31,"((AEGIS LOGISTICS LTD., U),)"
...,...,...,...
181092,P00118177,2023-03-31,"((HIGHWAYS INFRASTRUCTURE TRUST, U),)"
181093,P00118177,2024-03-31,"((HIGHWAYS INFRASTRUCTURE TRUST, U),)"
181094,P00118252,2024-03-31,"((MAX ESTATES LTD., U), (MAX ESTATES LTD., U))"
181095,P00118305,2024-03-31,"((EPACK DURABLE LTD., U),)"


### 1 COMP COUNT 1

In [54]:
pastDirect2["AsOnDate"] = pd.to_datetime(pastDirect2["AsOnDate"], format = "%Y-%m-%d")

# Creating a comp count only variables dataframe
compCountPre = pastDirect2.merge( currentDirectA2, on = ['Person Code', 'AsOnDate'] )
compCountPre2 = compCountPre.merge( currentDirectAB2, on = ['Person Code', 'AsOnDate'] )

In [55]:
# creating a tuple of Past Directorships

compCountPre2['PastSiezedDirectA'] = compCountPre2.progress_apply(
    lambda row: tuple(x for x in row['AllPastDirect'] if x not in row['CurrDirectA']), axis = 1)

compCountPre2['PastSiezedDirectAB'] = compCountPre2.progress_apply(
    lambda row: tuple(x for x in row['AllPastDirect'] if x not in row['CurrDirectAB']), axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 181097/181097 [00:03<00:00, 56022.58it/s]


In [56]:
compCount = compCountPre2.copy()

In [57]:
# Count Other Past Directorships A

compCount['CountOtherPastUnclearA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectA'] if x[1]=='U'), axis = 1)

compCount['CountOtherPastIndepA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectA'] if x[1]=='Y'), axis = 1)

compCount['CountOtherPastNonIndepA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectA'] if x[1]=='N'), axis = 1)

compCount['CompCountOtherPastTotalA'] = compCount['CountOtherPastUnclearA']+compCount['CountOtherPastIndepA']+compCount['CountOtherPastNonIndepA']

# Count Other Past Directorships AB

compCount['CountOtherPastUnclearAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectAB'] if x[1]=='U'), axis = 1)

compCount['CountOtherPastIndepAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectAB'] if x[1]=='Y'), axis = 1)

compCount['CountOtherPastNonIndepAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectAB'] if x[1]=='N'), axis = 1)

compCount['CompCountOtherPastTotalAB'] = compCount['CountOtherPastUnclearAB']+compCount['CountOtherPastIndepAB']+compCount['CountOtherPastNonIndepAB']

100%|███████████████████████████████████████████████████████████████████████| 181097/181097 [00:02<00:00, 82850.69it/s]


In [58]:
# Count Current Directorships A

compCount['CountCurrUnclearA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectA'] if x[1]=='U'), axis = 1)

compCount['CountCurrIndepA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectA'] if x[1]=='Y'), axis = 1)

compCount['CountCurrNonIndepA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectA'] if x[1]=='N'), axis = 1)

compCount['CompCountCurrTotalA'] = compCount['CountCurrUnclearA']+compCount['CountCurrIndepA']+compCount['CountCurrNonIndepA']

# Count Current Directorships AB

compCount['CountCurrUnclearAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectAB'] if x[1]=='U'), axis = 1)

compCount['CountCurrIndepAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectAB'] if x[1]=='Y'), axis = 1)

compCount['CountCurrNonIndepAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectAB'] if x[1]=='N'), axis = 1)

compCount['CompCountCurrTotalAB'] = compCount['CountCurrUnclearAB']+compCount['CountCurrIndepAB']+compCount['CountCurrNonIndepAB']

100%|███████████████████████████████████████████████████████████████████████| 181097/181097 [00:02<00:00, 89679.21it/s]


In [59]:
compCount

,Person Code,AsOnDate,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB
0,P00000005,2006-03-31,(),"((AEGIS LOGISTICS LTD., U),)","((AEGIS LOGISTICS LTD., U),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
1,P00000005,2007-03-31,"((AEGISCHEM, U),)","((AEGIS LOGISTICS LTD., U),)","((AEGIS LOGISTICS LTD., U),)","((AEGISCHEM, U),)","((AEGISCHEM, U),)",1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1
2,P00000005,2008-03-31,"((AEGISCHEM, U),)","((AEGIS LOGISTICS LTD., U),)","((AEGIS LOGISTICS LTD., U),)","((AEGISCHEM, U),)","((AEGISCHEM, U),)",1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1
3,P00000005,2009-03-31,"((AEGISCHEM, U),)","((AEGIS LOGISTICS LTD., U),)","((AEGIS LOGISTICS LTD., U),)","((AEGISCHEM, U),)","((AEGISCHEM, U),)",1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1
4,P00000005,2010-03-31,"((AEGISCHEM, U),)","((AEGIS LOGISTICS LTD., U),)","((AEGIS LOGISTICS LTD., U),)","((AEGISCHEM, U),)","((AEGISCHEM, U),)",1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181092,P00118177,2023-03-31,(),"((HIGHWAYS INFRASTRUCTURE TRUST, U),)","((HIGHWAYS INFRASTRUCTURE TRUST, U),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
181093,P00118177,2024-03-31,"((HIGHWAYS, U),)","((HIGHWAYS INFRASTRUCTURE TRUST, U),)","((HIGHWAYS INFRASTRUCTURE TRUST, U),)","((HIGHWAYS, U),)","((HIGHWAYS, U),)",1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1
181094,P00118252,2024-03-31,(),"((MAX ESTATES LTD., U),)","((MAX ESTATES LTD., U), (MAX ESTATES LTD., U))",(),(),0,0,0,0,0,0,0,0,1,0,0,1,2,0,0,2
181095,P00118305,2024-03-31,(),"((EPACK DURABLE LTD., U),)","((EPACK DURABLE LTD., U),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1


### 2 MERGE WITH MAIN 3

In [60]:
mainDirect3 = mainDirect2.merge( compCount, on = ["Person Code", "AsOnDate"] )
mainDirect3

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaT,nan,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0,(),"((3M INDIA LTD., U),)","((3M INDIA LTD., U),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,3289.0,(),"((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...","((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5
2,ASHOKLEY,ASHOK LEYLAND LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2000-07-27,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR D J BALAJI RAO, IS A FORMER DEPUTY MANAGING...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,2073.0,0,0,2073.0,2438.0,0,0,2438.0,2073.0,3289.0,(),"((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...","((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5
3,BAJAJHLDNG,BAJAJ HOLDINGS & INVESTMENT LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,1997-03-29,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,na

## DIRECTOR ROOKIE CLASSIFICATION AND FURTHER VARIABLES

In [1]:
mainDirect3.loc[ mainDirect3['YOEmax'] < 365*3, 'Rookie'] = 'Y'
mainDirect3.loc[ mainDirect3['YOEmax'] >= 365*3, 'Rookie'] = 'N'

mainDirect3

NameError: name 'mainDirect3' is not defined

In [62]:
mainDirect3['IsIndep'] = np.where( mainDirect3['Indep'] == 'Y', 1, 0)
mainDirect3['IsNonIndep'] =  np.where( mainDirect3['Indep'] == 'Y', 0, 1)

mainDirect3['IsRookie'] = np.where( mainDirect3['Rookie'] == 'Y', 1, 0)
mainDirect3['IsNonRookie'] = np.where( mainDirect3['Rookie'] == 'Y', 0, 1)

In [63]:
mainDirect3['Position on Board'].value_counts()

Position on Board
NON-EXECUTIVE DIRECTOR                                                                                       87320
nan                                                                                                          71258
EXECUTIVE DIRECTOR                                                                                            8983
EXECUTIVE DIRECTOR, PROMOTER-DIRECTOR                                                                         5975
NON-EXECUTIVE DIRECTOR, PROMOTER-DIRECTOR                                                                     5916
                                                                                                             ...  
CHAIRPERSON, MANAGING DIRECTOR, NOMINEE DIRECTOR OF MINISTRY OF POWER, GOVT.OF INDIA, EXECUTIVE DIRECTOR         1
NOMINEE DIRECTOR OF MINISTRY OF POWER, GOVT.OF INDIA, PART-TIME OFFICIAL DIRECTOR, NON-EXECUTIVE DIRECTOR        1
DIRECTOR-HUMAN RESOURCE, EXECUTIVE DIRECTOR                   

In [64]:
mainDirect3['Position on Board'] = mainDirect3['Position on Board'].astype(str)
mainDirect3['IsCeoMD'] = np.where( 
    mainDirect3['Position on Board'].str.contains(('CEO|CHIEF EXECUTIVE OFFICER|MANAGING DIRECTOR|MD'), case = False, na = False ) , 1,0)

In [65]:
mainDirect3['IsCeoMD'].value_counts()

IsCeoMD
0    207459
1     21096
Name: count, dtype: int64

In [66]:
mainDirect3['IsChairman'] = np.where( 
    mainDirect3['Position on Board'].str.contains(('CHAIRMAN|CHAIR PERSON|CHAIRPERSON|CHAIR'), case = False, na = False ) , 1,0)

mainDirect3['IsChairman'] = np.where( 
    mainDirect3['Position on Board'].str.contains(('DY. CHAIRPERSON|DY. CHAIRMAN|DY.CHAIRPERSON|VICE CHAIRPERSON|VICE-CHAIRPERSON|VICE -CHAIRPERSON|VICE- CHAIRPERSON|VICE - CHAIRPERSON|VICE CHAIRMAN|VICE-CHAIRMAN|VICE -CHAIRMAN|VICE- CHAIRMAN|VICE - CHAIRMAN'), case = False, na = False ) , 0, mainDirect3['IsChairman'])

In [67]:
mainDirect3['IsChairman'].value_counts()

IsChairman
0    209356
1     19199
Name: count, dtype: int64

In [68]:
mainDirect3['IsPromoterClassification'] = np.where( mainDirect3['Promoter Director (Yes/No)'] == 'YES', 1, 0)
mainDirect3['IsPromoterClassification'].value_counts()

IsPromoterClassification
0    194101
1     34454
Name: count, dtype: int64

In [69]:
mainDirect3['IsPromoterBoard'] = np.where(
    mainDirect3['Position on Board'].str.contains( ('PROMOTER'), case = False, na = False), 1, 0)
mainDirect3['IsPromoterBoard'].value_counts()

IsPromoterBoard
0    195305
1     33250
Name: count, dtype: int64

In [70]:
mainDirect3['IsPromoter'] = np.where( (mainDirect3['IsPromoterClassification'] == 1) | (mainDirect3['IsPromoterBoard'] == 1) , 1, 0)
mainDirect3['IsPromoter'].value_counts()

IsPromoter
0    192391
1     36164
Name: count, dtype: int64

In [71]:
mainDirect3['IsDualityChairmanMD'] = np.where( (mainDirect3['IsCeoMD'] == 1) & (mainDirect3['IsChairman'] == 1) , 1, 0)
mainDirect3['IsDualityChairmanMD'].value_counts()

IsDualityChairmanMD
0    221205
1      7350
Name: count, dtype: int64

In [72]:
mainDirect3['IsFamilyManaged'] = np.where( (mainDirect3['IsCeoMD'] == 1) & (mainDirect3['IsPromoter'] == 1) , 1, 0)
mainDirect3['IsFamilyManaged'].value_counts()

IsFamilyManaged
0    214748
1     13807
Name: count, dtype: int64

In [73]:
mainDirect3['IsFamilyChairman'] = np.where( (mainDirect3['IsPromoter'] == 1) & (mainDirect3['IsChairman'] == 1) , 1, 0)
mainDirect3['IsFamilyChairman'].value_counts()

IsFamilyChairman
0    216626
1     11929
Name: count, dtype: int64

In [74]:
mainDirect3['IsFamilyChairmanAndCEO'] = np.where( (mainDirect3['IsFamilyChairman'] == 1) & (mainDirect3['IsFamilyManaged'] == 1) , 1, 0)
mainDirect3['IsFamilyChairmanAndCEO'].value_counts()

IsFamilyChairmanAndCEO
0    222593
1      5962
Name: count, dtype: int64

In [75]:
mainDirect3['IsRookieIndep'] = np.where( (mainDirect3['IsRookie'] == 1) & (mainDirect3['IsIndep'] == 1) , 1, 0)
mainDirect3['IsRookieIndep'].value_counts()

IsRookieIndep
0    204999
1     23556
Name: count, dtype: int64

In [76]:
mainDirect3['IsRookieNonIndep'] = np.where( (mainDirect3['IsRookie'] == 1) & (mainDirect3['IsIndep'] == 0) , 1, 0)
mainDirect3['IsRookieNonIndep'].value_counts()

IsRookieNonIndep
0    191014
1     37541
Name: count, dtype: int64

In [77]:
mainDirect3['IsNonRookieIndep'] = np.where( (mainDirect3['IsRookie'] == 0) & (mainDirect3['IsIndep'] == 1) , 1, 0)
mainDirect3['IsNonRookieIndep'].value_counts()

IsNonRookieIndep
0    169303
1     59252
Name: count, dtype: int64

In [78]:
mainDirect3['IsNonRookieNonIndep'] = np.where( (mainDirect3['IsRookie'] == 0) & (mainDirect3['IsIndep'] == 0) , 1, 0)
mainDirect3['IsNonRookieNonIndep'].value_counts()

IsNonRookieNonIndep
0    120349
1    108206
Name: count, dtype: int64

In [79]:
mainDirect3['IsFemale'] = np.where( (mainDirect3['Gender'] == 'F'), 1,0)
mainDirect3['IsFemale'].value_counts()

IsFemale
0    200842
1     27713
Name: count, dtype: int64

In [80]:
now = pd.Timestamp('now')

mainDirect3['Date of Birth'] = pd.to_datetime(mainDirect3['Date of Birth'], format = '%Y-%m-%d')

mainDirect3['Age'] = np.floor((now - mainDirect3['Date of Birth']).dt.days / 365).astype('Int64')

In [81]:
mainDirect3['TenureInYearsIndep'] = mainDirect3['OpBalIndep']/365
mainDirect3

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaT,nan,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0,(),"((3M INDIA LTD., U),)","((3M INDIA LTD., U),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,64,0.0
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,3289.0,(),"((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...","((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,85,0.0
2,ASHOKLEY,ASHOK LEYLAND LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2000-07-27,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR D J BALAJI RAO, IS A FORMER DEPUTY MANAGING...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,2073.0,0,0,2073.0,2438.0,0,0,2438.0,2073.0,3289.0,(),"((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...","((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,85,0.0
3,BAJAJHLDNG,BAJAJ HOLDINGS & INVESTMENT LTD.,200

In [82]:
mainDirect3['TenureInYearsTotal'] = mainDirect3['OpBalTotal']/365
mainDirect3

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep,TenureInYearsTotal
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaT,nan,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0,(),"((3M INDIA LTD., U),)","((3M INDIA LTD., U),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,64,0.0,3.687671
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,3289.0,(),"((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...","((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,85,0.0,4.263014
2,ASHOKLEY,ASHOK LEYLAND LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2000-07-27,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR D J BALAJI RAO, IS A FORMER DEPUTY MANAGING...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,2073.0,0,0,2073.0,2438.0,0,0,2438.0,2073.0,3289.0,(),"((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...","((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,85,0.0,5.679452
3,B

In [83]:
mainDirect3['IsFirstTermIndep'] = np.where( (mainDirect3['IsIndep'] == 1) & (mainDirect3['OpBalTotal'] < 365*5), 1, 0)

In [84]:
mainDirect3['IsRetires5y'] = np.where((mainDirect3['OpBalTotal'] <= 365*5) & (mainDirect3['OpBalTotal'] > 365*4), 1, 0)

In [85]:
mainDirect3['IsTermLimitRetirement'] = np.where((mainDirect3['OpBalTotal'] <= 365*10) & (mainDirect3['OpBalTotal'] > 365*9), 1, 0)

In [86]:
mainDirect3['IsDefaultTerm'] = np.where((mainDirect3['OpBalTotal'] > 365*10), 1, 0)

In [87]:
mainDirect3['AsOnYear'] = pd.DatetimeIndex(mainDirect3['AsOnDate']).year

In [88]:
def Retirement(frame):

#aight shiv, get only the year from the date store it as integer. and check what is the max year(int) and put turnover =1 for the previous year or year-1
#call apply by grouping person code and company

#check if year+1 == exists at year where isretire5y = 1, if not, put hasretires5 at year = 1
#check if year+1 == exists at year where IsTermLimitRetirement = 1, if not, put hasTermLimitRetirement at year = 1

    years = frame['AsOnYear'].astype(int).tolist()

    def Conditions(row):
        turnover = 0
        HasRetires5y = 0
        HasTermLimitRetirement = 0
        year = int(row['AsOnYear'])

        if (year+1 not in years):
            turnover = 1
            if (row['IsRetires5y'] == 1):
                HasRetires5y = 1
            if (row['IsTermLimitRetirement'] == 1):
                HasTermLimitRetirement = 1

        return turnover, HasRetires5y, HasTermLimitRetirement

    frame['IsTurnOver'], frame['HasRetires5y'], frame['HasTermLimitRetirement'] = zip(*frame.apply(Conditions, axis = 1))
    return frame

In [89]:
retire = mainDirect3.loc[ :, ['Person Code', 'Symbol', 'AsOnYear', 'IsRetires5y', 'IsTermLimitRetirement']]

In [90]:
retire2 = retire.groupby(['Person Code', 'Symbol']).progress_apply(Retirement)

100%|███████████████████████████████████████████████████████████████████████████| 43722/43722 [04:40<00:00, 156.10it/s]


In [91]:
retire2

Person Code      Symbol  AsOnYear  IsRetires5y  \
Person Code Symbol                                                             
P00000005   AEGISCHEM  222      P00000005   AEGISCHEM      2006            0   
                       6606     P00000005   AEGISCHEM      2007            0   
                       14576    P00000005   AEGISCHEM      2008            0   
                       23778    P00000005   AEGISCHEM      2009            0   
                       33435    P00000005   AEGISCHEM      2010            0   
...                                   ...         ...       ...          ...   
P00118177   HIGHWAYS   201979   P00118177    HIGHWAYS      2023            0   
                       218587   P00118177    HIGHWAYS      2024            0   
P00118252   MAXESTATES 221991   P00118252  MAXESTATES      2024            0   
P00118305   EPACK      216719   P00118305       EPACK      2024            0   
P00118306   EPACK      216714   P00118306       EPACK      2024            1   

                               IsTermLimitRetirement  IsTurnOver  \
Person Code Symbol                                                 
P00000005   AEGISCHEM  222                         0           0   
                       6606                        0           0   
                       14576                       0           0   
                       23778                       0           0   
                       33435                       0           0   
...                                              ...         ...   
P00118177   HIGHWAYS   201979                      0           0   
                       218587                      0           1   
P00118252   MAXESTATES 221991                      0           1   
P00118305   EPACK      216719                      0           1   
P00118306   EPACK      216714                      0           1   

                               HasRetires5y  HasTermLimitRetirement  
Person Code Symbol                                                   
P00000005   AEGISCHEM  222                0                       0  
                       6606               0                       0  
                       14576              0                       0  
                       23778              0                       0  
                       33435              0                       0  
...                                     ...                     ...  
P00118177   HIGHWAYS   201979             0                       0  
                       218587             0                       0  
P00118252   MAXESTATES 221991             0                       0  
P00118305   EPACK      216719             0                       0  
P00118306   EPACK      216714             1                       0  

[228555 rows x 8 columns]

In [92]:
retire3 = retire2.reset_index( drop = True)
retire3

,Person Code,Symbol,AsOnYear,IsRetires5y,IsTermLimitRetirement,IsTurnOver,HasRetires5y,HasTermLimitRetirement
0,P00000005,AEGISCHEM,2006,0,0,0,0,0
1,P00000005,AEGISCHEM,2007,0,0,0,0,0
2,P00000005,AEGISCHEM,2008,0,0,0,0,0
3,P00000005,AEGISCHEM,2009,0,0,0,0,0
4,P00000005,AEGISCHEM,2010,0,0,0,0,0
...,...,...,...,...,...,...,...,...
228550,P00118177,HIGHWAYS,2023,0,0,0,0,0
228551,P00118177,HIGHWAYS,2024,0,0,1,0,0
228552,P00118252,MAXESTATES,2024,0,0,1,0,0
228553,P00118305,EPACK,2024,0,0,1,0,0


In [93]:
mainDirect4 = mainDirect3.merge( retire3, on = ['Person Code', 'AsOnYear', 'Symbol', "IsRetires5y", "IsTermLimitRetirement"])

In [94]:
mainDirect4

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep,TenureInYearsTotal,IsFirstTermIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,AsOnYear,IsTurnOver,HasRetires5y,HasTermLimitRetirement
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaT,nan,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0,(),"((3M INDIA LTD., U),)","((3M INDIA LTD., U),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,64,0.0,3.687671,0,0,0,0,2006,0,0,0
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,3289.0,(),"((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...","((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,85,0.0,4.263014,0,1,0,0,2006,0,0,0
2,ASHOKLEY,ASHOK LEYLAND LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2000-07-27,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR D J BALAJI RAO, IS A FORMER DEPUTY MANAGING...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,2073.0,0,0,2073.0,2438.0,0,0,2438.0,2073.0,3289.0,(),"((ASHOK LEYLAND LTD., U), (3M INDIA LTD

In [95]:
mainDirect4['IsTurnOverIndep'] = np.where((mainDirect4['IsTurnOver'] == 1) & (mainDirect4['IsIndep'] == 1), 1, 0)

In [96]:
mainDirect4

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep,TenureInYearsTotal,IsFirstTermIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,AsOnYear,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaT,nan,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0,(),"((3M INDIA LTD., U),)","((3M INDIA LTD., U),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,64,0.0,3.687671,0,0,0,0,2006,0,0,0,0
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,3289.0,(),"((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...","((ASHOK LEYLAND LTD., U), (3M INDIA LTD., U), ...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,85,0.0,4.263014,0,1,0,0,2006,0,0,0,0
2,ASHOKLEY,ASHOK LEYLAND LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2000-07-27,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaT,nan,"MR D J BALAJI RAO, IS A FORMER DEPUTY MANAGING...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaT,NaT,U,1,0,0,2073.0,0,0,2073.0,2438.0,0,0,2438.0,2073.0,3289.0,(),"((ASHOK LEYLAND LTD

In [97]:
mainDirect4.to_csv("Main_Director_complete v160225.csv")